# Data mining of semiconductor materials

This notebook will initially serve as an example in how to extract information of the database Materials Project [1]. 
In total, there are 126335 entries in the materials project whereas 48644 of them, roughly 39%, are deemed to be structurally similar from an experimental ICSD entry according to pymatgen's StructureMatcher algorithm. Additionally, a total of 65783 entries have been calculated to have a band gap larger than $0.1$ eV. These two have an overlap of 25271 entries, which is our starting point. 

The notebook will consist of 3 stages, and is strongly inspired from Ferrenti et al [2]. However, we diverge at stage 3 where we have moved the extraction of other databases out before we attend this notebook.

Additionally, we will use the datamining process to find fitting and unfitting candidates. The process can be described by following some given criteria for good candidates, resulting in the label $1$. Then, we will use the complete opposite criteria to find unfitted candidates, resulting in the label $0$. 
 
 
### Contents
  #### Fitting candidates
    - Stage 1
        - $50\% + l = 0$ isotopes, with some exceptions
        - Calculated non-magnetic 
        - Has experimental ICSD entry
        - Crystallize in non-polar space groups
    - Stage 2
        - No Th, U, Cd, Hg
        - No noble gases or rare-earth elements
    - Stage 3
        - bandgap restriction
        
  #### Unfitted candidates
    - Stage 1
        - $50\% + l != 0$ isotopes
        - Calculated magnetic
        - Has experimental ICSD entry
        - Crystallize in polar space groups
    - Stage 2
        - Include Th, U, Cd, Hg
        - Include noble gases or rare-earth elements    
    - Stage 3
        - bandgap restriction
        - Summarize bandgaps and other properties

[1] Ong, S. P.; Cholia, S.; Jain, A.; Brafman, M.; Gunter, D.; Ceder, G.; 
Persson, K. a. The Materials Application Programming Interface (API): A 
simple, flexible and efficient API for materials data based on
REpresentational State Transfer (REST) principles, Comput. Mater. Sci.,
2015, 97, 209–215. doi:10.1016/j.commatsci.2014.10.037.

[2] Ferrenti, A.M., de Leon, N.P., Thompson, J.D. et al. Identifying candidate hosts for quantum defects via data mining. npj Comput Mater 6, 126 (2020). https://doi.org/10.1038/s41524-020-00391-7

Initially, we start of with some imports. 

In [1]:
# Optional: Load the "autoreload" extension so that code can change
%load_ext autoreload

#OPTIONAL: Always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [2]:
from pathlib import Path
data_dir = Path.cwd().parent.parent / "data" 
print("Current data directory {}".format(data_dir))

Current data directory /home/oliver/Dokumenter/masterprosjekt/predicting-solid-state-qubit-candidates/data


In [ ]:
# Tools for query of data
from pymatgen import MPRester, Composition

# Finding correct use of polar groups
from src.data.utils import polarGroupUsedInMP, sortByMPID, filterIDs
from src.visualization import visualize

# pandas
import pandas as pd
import numpy as np

from tqdm import tqdm

# Ignore warnings from nan-values in  
np.warnings.filterwarnings('ignore')

import os
# Find and store all API-keys that are stored as environment variables .env in root folder
from dotenv import find_dotenv, load_dotenv
key_status = load_dotenv(find_dotenv())

# Private keys. If not present, add your own secret keys here
if (key_status):
    MAPI_KEY = os.getenv("MAPI_KEY")
    CAPI_KEY = os.getenv("CAPI_KEY")
else: 
    MAPI_KEY = None
    CAPI_KEY = None

In [ ]:
InsertApproach = "02-determined-approach"

# Fitted candidates
## Stage 1
Thereafter, we need to find out what elements we want to include. The article mentioned above have some strict restrictions that are as follows: 
- $50\% + l = 0$ isotopes
- Calculated non-magnetic
- Has experimental ICSD entry
- Crystallize in non-polar space groups

This stage will be done entirely through a query to pymatgen, however, the restrictions need to be set properly first. 

In [ ]:
spin_zero_isotopes = [
    "H", "Li", "Be", "F", "Na", "Cl", "K", "Sc",
    "V", "Mn", "Co", "Cu", "Br", "Rb", "Y", "Nb", "Tc", 
    "Rh", "Ag", "In", "Sb", "I", "Cs", "Lu", "Ta", "Re", "Ir", "Au",
    "Tl", "Bi", "Po", "At", "Rn", "Fr", "Ra", "La","Pr", "Pm", "Eu", "Tb", "Ho",
    "Tm", "Ac", "Pa", "Np", "Pu", "Am"]  #(47)

#exceptions : 
# Al, P, Ga, As, B, N

print("Number of excluded periodic-elements from isotopes: {}".format(len(spin_zero_isotopes)))

In [ ]:
polar_spacegroups = polarGroupUsedInMP()

### Query

In [ ]:
with MPRester(MAPI_KEY) as mpr:
    
    criteria = {'elements':{"$nin": spin_zero_isotopes}, #not included
                    'icsd_ids': {'$gte': 0}, #All compounds deemed similar to a structure in ICSD
                    "magnetic_type": {"$eq": "NM"}, #non-magnetic
                    "spacegroup.number": {"$nin": polar_spacegroups}
                    }

    props = ["material_id","full_formula", "spacegroup", "band_gap", "e_above_hull"]
    fitted_entries = pd.DataFrame(mpr.query(criteria=criteria, properties=props))    
        
print("Number of entries after query: {}".format(len(fitted_entries)))

In [ ]:
def polarGroupUnitTest(entries):
    # Unit tests for polar groups. 
    ###############################
    # Remove all entries with polar space groups
    exclude_polar_space_groups =  {
         "triclinic":   ["1"], 
         "monoclinic":  ["2", "m"],
         "orthorhombic":["mm2"],
         "tetragonal":  ["4", "4mm"],
         "trigonal":    ["3", "3m"],
         "hexagonal":   ["6", "6mm"]
        }

    #remove polar spacegroups
    deleteEntries = []
    for i, entry in entries.iterrows():
        #else: 
        if (entry["spacegroup"]["crystal_system"]) in exclude_polar_space_groups.keys():
            if entry["spacegroup"]["point_group"] in exclude_polar_space_groups[entry["spacegroup"]["crystal_system"]]:
                deleteEntries.append(i)
                
    #Every delete will return a smaller dict
    numberDeleted = 0
    for deleteEntry in deleteEntries: 
        del entries[deleteEntry-numberDeleted]
        numberDeleted += 1
    if numberDeleted > 0:
        print("Test not passed, polar groups could be wrong")
    else:
        print("Polar group test passed.")
polarGroupUnitTest(fitted_entries)

# Stage 2

In [ ]:
lowerBandGapLimit = 1.5

fitted_entries = fitted_entries[fitted_entries["band_gap"] >= lowerBandGapLimit]
fitted_entries

# Stage 3

In [ ]:
fitted_entries = fitted_entries[fitted_entries["e_above_hull"]<0.2].reset_index(drop=True)
fitted_entries

In [ ]:
fitted_entries["candidate"] = np.ones(len(fitted_entries))

# Unfitted candidates
## Stage 1

- Calculated magnetic
- Has experimental ICSD entry
- Crystallize in polar space groups

In [ ]:
""" For total contrast in stages from good to bad candidates, add these elements.
spin_isotopes = ["He","C", "O", "Ne", "Mg", "Si", "S", "Ar",
                "Ca", "Ti", "Cr", "Fe", "Ni", "Zn", "Ge", "Se",
                "Se", "Kr", "Sr", "Zr", "Mo", "Ru", "Pd", "Cd",
                "Sn", "Te", "Xe", "Ba", "Hf", "W", "Os", "Pt",
                "Hg", "Pb", "Ce", "Nd", "Sm", "Gd", "Dy", "Er",
                "Yb", "Th", "U"] #43

#Not include the following elements:
include_elements = [ 
    "Th", "U", "Cd", "Hg", #restriction nr 1 above (4)
    "He", "Ne", "Ar", "Kr", "Xe", "Rn", "Og", #no noble gases (7)
    "Sc", "Y", "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd",
     "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu"]
    # No rare-earth elements (17)

indices = []
for i, spin in enumerate(spin_isotopes): 
    for ele in include_elements:
        if spin == ele:
            indices.append(i)
            
for i in sorted(indices, reverse=True):
    del spin_isotopes[i]

len(spin_isotopes)
"""

### Query

In [ ]:

with MPRester(MAPI_KEY) as mpr:
    
    criteria = {#'elements':{"$nin": spin_isotopes}, #not included
                'icsd_ids': {'$gte': 0}, #All compounds deemed similar to a structure in ICSD
                "magnetic_type": {"$ne": "NM"}, #non-magnetic not equal
                "spacegroup.number": {"$in": polar_spacegroups}
                }

    props = ["material_id","full_formula", "spacegroup", "band_gap"]
    unfitted_entries = pd.DataFrame(mpr.query(criteria=criteria, properties=props))    
        
print("Number of entries after query: {}".format(len(unfitted_entries)))

# Stage 2

For consistency we are only looking at semiconductors. Thus, we will maintain a lower band gap limit for unfitted candidates, since the features we are generating are based on that a material has a band gap. 

In [ ]:
lowerBandGapLimit = 0.1

unfitted_entries = unfitted_entries[unfitted_entries["band_gap"] >= lowerBandGapLimit]
unfitted_entries

In [ ]:
unfitted_entries["candidate"] = np.zeros(len(unfitted_entries))

# Combine data and create training and test set

In this section we will combine the data we have extracted, both from the Generated Data notebook, and the labels from the previous dataMining section. 

In [ ]:
trainingTargets = pd.concat([fitted_entries,unfitted_entries]).reset_index(drop=True)

trainingTargets = sortByMPID(trainingTargets)
trainingTargets = filterIDs(trainingTargets)
trainingTargets

In [ ]:
drop_columns =  ["full_formula", "spacegroup", "band_gap", "e_above_hull"]
trainingTargets = trainingTargets.drop(drop_columns, axis=1)

In [ ]:
#Path(data_dir / InsertApproach / "processed").mkdir(parents=True, exist_ok=True)
#trainingTargets.to_pickle(data_dir / InsertApproach / "processed" / "trainingCombo.pkl")

# Distribution of entries in the data
How is the distribution of entries in the different features? 
## Elements

In [ ]:
trainingTargets

In [ ]:
generatedData = pd.read_pickle(data_dir / "interim" / "featurized" / "featurized-19-03-2021.pkl")
trainingTargets = trainingTargets[trainingTargets["material_id"].isin(generatedData["material_id"])]

In [ ]:
testSet = (
    trainingTargets.merge(generatedData, 
              on='material_id', 
              how='outer', 
              indicator=True)
    .query('_merge != "both"')
    .drop(columns='_merge')
)
trainingSet = (
    trainingTargets.merge(generatedData,
                on="material_id",
                indicator=False,
                how="left",
                suffixes=(False, False))
)

In [ ]:
print(testSet.shape)
print(trainingSet.shape)

In [ ]:
visualize.make_parallel_coordinate_matplot(trainingSet, InsertApproach)

In [ ]:
print("The amount of good qubit host candidates: {}, or {:0.4f}%".format(trainingSet[trainingTargets.values==1].shape[0], trainingSet[trainingTargets.values==1].shape[0]/trainingSet.shape[0]))
print("The amount of bad qubit host candidates: {}, or {:0.4f}%".format(trainingSet[trainingTargets.values==0].shape[0], trainingSet[trainingTargets.values==0].shape[0]/trainingSet.shape[0]))

In [ ]:
print("Information about elements in compounds are given in the interval:")
print(np.where(generatedData.columns=="H")[0][0],np.where(generatedData.columns=="Pu")[0][0])
# plotting 
import plotly.graph_objects as go
elements = generatedData.columns[
    np.where(generatedData.columns=="H")[0][0]:np.where(generatedData.columns=="Pu")[0][0]
].values
print(len(elements))
fig = go.Figure( 
    layout = go.Layout (
        title=go.layout.Title(text="Distribution of elements in training data"),
        yaxis=dict(title='Number'),
        xaxis=dict(title='Elements'),
        xaxis_tickangle=-45
        )
    )

width_plotly = 548.1896533333334

fittedCounts = trainingSet[elements][trainingTargets.values==1].fillna(0).astype(bool).sum(axis=0)
unFittedCounts   = trainingSet[elements][trainingTargets.values==0].fillna(0).astype(bool).sum(axis=0)

fig.add_traces(go.Bar(name = "Fitted candidates",#prettyNames[i], 
                        x = trainingSet[elements].columns,
                        y = fittedCounts,
                        text = trainingSet[elements].columns,
                        )
                )
fig.add_traces(go.Bar(name = "Unfitted candidates",#prettyNames[i], 
                        x = trainingSet[elements].columns,
                        y = unFittedCounts,
                        text = trainingSet[elements].columns,
                        )
                )
fig.update_layout(
                    {"plot_bgcolor": "rgba(0, 0, 0, 0)",
                       "paper_bgcolor": "rgba(0, 0, 0, 0)",
                      },
                      font=dict(
                        family="Palatino",
                        color="Black",
                        size=12),
                      autosize=False,
                      width=width_plotly*1.5,
                      height=width_plotly/2,
                     )
fig.write_image(str(Path.cwd().parent.parent / 
                                    "reports" / "figures"  / "buildingFeatures" 
                                    / "determined-approach-normalized-elements-histogram.pdf"))
fig.show()

In [ ]:
fig = go.Figure( 
    layout = go.Layout (
        title=go.layout.Title(text="Normal adjusted counts of elements in training data"),
        yaxis=dict(title='Normal adjusted counts'),
        xaxis=dict(title='Elements'),
        xaxis_tickangle=-45
        )
    )

fig.add_traces(
    go.Bar(name = "Fitted candidates",#prettyNames[i], 
                   x =    trainingSet[elements].columns,
                   y =    fittedCounts/np.sum(fittedCounts),
                   text = trainingSet[elements].columns,
                    )
                )
fig.add_traces(go.Bar(name = "Unfitted candidates",#prettyNames[i], 
                    x = trainingSet[elements].columns,
                    y = unFittedCounts/np.sum(unFittedCounts),
                    text = trainingSet[elements].columns,
                    )
                )
fig.show()

## Known candidates 

How are the known candidates already doing in the datamining process? SiC, etc. 

In [ ]:
feat = ["material_id","MP_Eg","pretty_formula","full_formula"]

#generatedData[feat][generatedData["pretty_formula"]=="SiC"]

In [ ]:
feat = ["material_id","MP_Eg","pretty_formula","full_formula", "candidate"]

trainingSet[feat][(trainingSet["pretty_formula"]=="SiC")]

In [ ]:
trainingSet[feat][(trainingSet["pretty_formula"]=="C")]

## Combine training targets with preprocessed data

In [ ]:
PCAGeneratedData = pd.read_pickle(data_dir / "processed" / "processedData.pkl")
PCAGeneratedData.shape

In [ ]:
trainingSet = (
    trainingTargets.merge(PCAGeneratedData,
                on="material_id",
                indicator=False,
                how="left",
                suffixes=(False, False))
)
trainingSet.shape

In [ ]:
# Drop new candidates that might have been added to MP after featurization 
trainingSet = trainingSet[trainingSet["material_id"].isin(PCAGeneratedData["material_id"])]
trainingSet.shape

In [ ]:
testSet = (
    trainingTargets.merge(PCAGeneratedData, 
              on='material_id', 
              how='outer', 
              indicator=True)
    .query('_merge != "both"')
    .drop(columns='_merge')
)
testSet.shape

In [ ]:
# Drop new candidates that might have been added to MP after featurization 
testSet = testSet[testSet["material_id"].isin(PCAGeneratedData["material_id"])]
testSet.shape

# Change column order
trainingSet.insert(1, 'full_formula', trainingSet.pop("full_formula"))
testSet    .insert(1, 'full_formula', testSet    .pop("full_formula"))
testSet    .insert(1, 'pretty_formula', testSet    .pop("pretty_formula"))

## Write to file

In [ ]:
trainingSet = trainingSet.drop(["pretty_formula"], axis=1)
trainingTarget = trainingSet.pop("candidate")

Path(data_dir / InsertApproach / "processed").mkdir(parents=True, exist_ok=True)
trainingSet   .to_pickle(data_dir / InsertApproach /"processed" / "trainingData.pkl")
trainingTarget.to_pickle(data_dir / InsertApproach / "processed" / "trainingTarget.pkl")
testSet       .to_pickle(data_dir / InsertApproach / "processed" / "testSet.pkl")

In [ ]:
testSet

In [ ]:
df = trainingSet.copy()
df["candidate"] = trainingTarget
df = df.groupby('candidate').apply(lambda s: s.sample(min(len(s), 300)))
visualize.plot_2d_pca(df, df["candidate"], InsertApproach)